https://www.youtube.com/watch?v=osv2csoHVAo&t=1s  
https://github.com/HongLabInc/HongLabLLM/blob/main/01_pretraining.ipynb

# Data
harry potter book - https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books?resource=download  
alice - https://www.kaggle.com/datasets/leelatte/alicetxt

In [1]:
"/mnt/d/Data/llm_data"

'/mnt/d/Data/llm_data'

In [2]:
import re


def clean_data(filr_dir, file_name):

    with open(f"{filr_dir}/{file_name}", "r", encoding="utf-8") as file:
        book_text = file.read()

    cleaned_data = re.sub(r"\n+", " ", book_text)  # 줄 바꿈을 하나의 공백으로 변경
    cleaned_data = re.sub(r"\s", " ", cleaned_data)  # 여러 공백을 하나의 공백으로 변경

    print(f"cleaned_{file_name, len(cleaned_data)} characters")  # 글자 수 출력

    save_dir = filr_dir[:-9]
    # save_file_name = file_name.split("/")[-1]

    with open(f"{save_dir}/cleaned/{file_name}", "w", encoding="utf-8") as file:
        file.write(cleaned_data)

In [3]:
import os

harry_potter_dir = "/mnt/d/Data/llm_data/harry_potter/original"
harry_potter_file = os.listdir(harry_potter_dir)

for file in harry_potter_file:
    clean_data(harry_potter_dir, file)

cleaned_('01 Harry Potter and the Sorcerers Stone.txt', 436001) characters
cleaned_('02 Harry Potter and the Chamber of Secrets.txt', 488771) characters
cleaned_('03 Harry Potter and the Prisoner of Azkaban.txt', 621137) characters
cleaned_('04 Harry Potter and the Goblet of Fire.txt', 1093670) characters
cleaned_('05 Harry Potter and the Order of the Phoenix.txt', 1489734) characters
cleaned_('06 Harry Potter and the Half-Blood Prince.txt', 982041) characters
cleaned_('07 Harry Potter and the Deathly Hallows.txt', 1133063) characters


# Tokenizer

In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = "Harry Potter was a wizard."

tokens = tokenizer.encode(text)

print(f"글자 수 : {len(text)}, 토큰 수 : {len(tokens)}")
print(tokens)  # 인코딩
print(tokenizer.decode(tokens))  # 디코딩

for t in tokens:
    print(f"{t}\t -> {tokenizer.decode([t])}")

글자 수 : 26, 토큰 수 : 6
[18308, 14179, 373, 257, 18731, 13]
Harry Potter was a wizard.
18308	 -> Harry
14179	 ->  Potter
373	 ->  was
257	 ->  a
18731	 ->  wizard
13	 -> .


In [5]:
for char in text:
    token_ids = tokenizer.encode(char)  # 한 글자씩 인코딩
    decoded = tokenizer.decode(token_ids)  # 한 글자씩 디코딩
    print(f"{char} -> {token_ids} -> {decoded}")
    # 공백, 마침표도 토큰화 됨

H -> [39] -> H
a -> [64] -> a
r -> [81] -> r
r -> [81] -> r
y -> [88] -> y
  -> [220] ->  
P -> [47] -> P
o -> [78] -> o
t -> [83] -> t
t -> [83] -> t
e -> [68] -> e
r -> [81] -> r
  -> [220] ->  
w -> [86] -> w
a -> [64] -> a
s -> [82] -> s
  -> [220] ->  
a -> [64] -> a
  -> [220] ->  
w -> [86] -> w
i -> [72] -> i
z -> [89] -> z
a -> [64] -> a
r -> [81] -> r
d -> [67] -> d
. -> [13] -> .


# Dataloader

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader


class LLM_Dataset(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        print(f"# of tokens in txt: {len(token_ids)}")

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]  # 입력 데이터의 두 번째 단어부터 마지막 다음 단어까지 출력 됨
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [7]:
with open("/mnt/d/Data/llm_data/harry_potter/cleaned/01 Harry Potter and the Sorcerers Stone.txt", "r", encoding="utf-8-sig") as file:
    txt = file.read()

dataset = LLM_Dataset(txt=txt, max_length=32, stride=4)

train_loader = DataLoader(dataset=dataset, batch_size=128, shuffle=True, drop_last=True)

# of tokens in txt: 117768


In [8]:
data_iter = iter(train_loader)

x, y = next(data_iter)

print(tokenizer.decode(x[0].tolist()))
print(tokenizer.decode(y[0].tolist()))

 you say?” his aunt snapped through the door. “Nothing, nothing…” Dudley’s birthday — how could he have forgotten?
 say?” his aunt snapped through the door. “Nothing, nothing…” Dudley’s birthday — how could he have forgotten? Harry


# Neural Network Model Define

In [ ]:
VOCAB_SIZE = tokenizer.n_vocab  # 50257 Tiktoken 사용시
# VOCAB_SIZE = len(tokenizer) # AutoTokenizer 사용 시
CONTEXT_LENGTH = 128  # Shortened context Length (orig : 1024)
EMB_DIM = 768  # Embedding dimension
NUM_HEADS = 12  # Number of attention heads
NUM_LAYERS = 12  # Number of Layers
DROP_RATE = 0.1  # Dropout rate
QKV_BIAS = False  # Query-key-value bias

In [10]:
import torch.nn as nn


class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()

        assert d_out % NUM_HEADS == 0, "d_out must be divisivle by n_heads"

        self.d_out = d_out
        self.head_dim = d_out // NUM_HEADS

        self.W_query = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_key = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_value = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(DROP_RATE)
        self.register_buffer("mask", torch.triu(torch.ones(CONTEXT_LENGTH, CONTEXT_LENGTH), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, NUM_HEADS, self.head_dim)
        queries = queries.view(b, num_tokens, NUM_HEADS, self.head_dim)
        values = values.view(b, num_tokens, NUM_HEADS, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec


class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var * self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super.__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.44715 * torch.pow(x, 3))))


class FeedForward(nn.Module):
    def __init__(self):
        super.__init__()
        self.layers = nn.Sequential(
            nn.Linear(EMB_DIM, 4 * EMB_DIM),
            GELU(),
            nn.Linear(4 * EMB_DIM, EMB_DIM),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self):
        super.__init__()
        self.att = MultiHeadAttention(d_in=EMB_DIM, d_out=EMB_DIM)
        self.ff = FeedForward()
        self.norm1 = LayerNorm(EMB_DIM)
        self.norm2 = LayerNorm(EMB_DIM)
        self.drop_shortcut = nn.Dropout(DROP_RATE)

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x


class GPTModel(nn.Module):
    def __init__(self):
        super.__init__()
        self.tok_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM)
        self.pos_emb = nn.Embedding(CONTEXT_LENGTH, EMB_DIM)
        self.drop_emb = nn.Dropout(DROP_RATE)

        self.trf_blocks = nn.Sequential(*[TransformerBlock() for _ in range(NUM_LAYERS)])
        self.final_norm = LayerNorm(EMB_DIM)
        self.out_head = nn.Linear(EMB_DIM, VOCAB_SIZE, bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)

        return logits

# Train